In [13]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
import pandas as pd
import os
import numpy as np
import time

#from hyphen import Hyphenator
#hyphe = Hyphenator('en_GB')
import pyphen
dic = pyphen.Pyphen(lang='en')

pd.set_option('display.max_rows', 255)
pd.set_option('display.max_columns', 100)

In [16]:
listFossilsGhost = ['FossilKabutops', 'FossilAerodactyl', 'MonGhost',]
listFossilsGhost_Capital = ['FOSSIL_KABUTOPS', 'FOSSIL_AERODACTYL', 'MON_GHOST',]
listFossilsGhost_Names = ['MissingNo.', 'MissingNo.', 'MissingNo.']
listFossilsGhost_Names_Capital = ['MISSINGNO.', 'MISSINGNO.', 'MISSINGNO.']


In [17]:
def printNCharacters(N, paddingChar=' '):
    stringHere = ''
    for idx in range(N):
        stringHere+=paddingChar
    return stringHere

In [18]:
def printOneInfoForOneMon(columnOfInterest, PokemonOfInterest, capitalizationMon='Name_allsmall'):
    return DfPkmnInfoFromExcel[columnOfInterest].tolist()[DfPkmnInfoFromExcel[capitalizationMon].tolist().index(PokemonOfInterest)]

def getOneColumn(columnOfInterest):
    return DfPkmnInfoFromExcel[columnOfInterest].tolist()

In [19]:
def readASMFileReturnLinesAndModifiedPath(asmPath, newSuffix='_fromExcel'):
    asmPathNew = asmPath[:-len('.asm')]+newSuffix+'.asm'

    try:
        file_in = open(asmPath, 'r')
    except:
        linesFromFile = []
    else:
        file_in = open(asmPath, 'r')
        linesFromFile = []
        for y in file_in.readlines():
            linesFromFile.append(y)
        
    return linesFromFile, asmPathNew

In [20]:
def lineifyDexEntries_OneLine(DexEntryWordByWord=['Hello', 'world'], lengthMax=18, debug=False, syllableBreaking=True):
    stringTemp = ''
    lengthTemp = len(stringTemp)
    idxTemp = 0

    keepAddingWords = True
    while keepAddingWords:
        if stringTemp=='':
            stringTempNext = stringTemp + DexEntryWordByWord[idxTemp]
        else:
            stringTempNext = stringTemp + ' ' + DexEntryWordByWord[idxTemp]

        # more than 18 chars
        if len(stringTempNext)>lengthMax:
            # check if just adding one space and two symbols is enough to go beyond limits:
            # previous string must have ended on a word, so a space is needed,
            # also if the word was one-letter long and fit, it would pass the test above,
            # so the best-case scenario is that the new word can be split from the first letter
            if len(stringTemp)+3>lengthMax:
                keepAddingWords = False
            else:
                # check if the next word can be split into syllables

                # used with hyphen and not pyphen
                #if syllableBreaking:
                #    syllablesTemp = hyphe.syllables(DexEntryWordByWord[idxTemp])
                #else:
                #    syllablesTemp = [DexEntryWordByWord[idxTemp]] # testing

                # used with hyphen and not pyphen
                #while syllablesTemp.count('-')>0:
                #    syllablesTemp.remove('-')

                # now used with pyphen                
                if syllableBreaking:
                    print('idxTemp =', idxTemp) if debug else _
                    print('DexEntryWordByWord[idxTemp] =', DexEntryWordByWord[idxTemp]) if debug else _
                    syllablesTemp = dic.inserted(DexEntryWordByWord[idxTemp])
                    print('syllablesTemp =', syllablesTemp) if debug else _
                else:
                    print('idxTemp =', idxTemp) if debug else _
                    print('DexEntryWordByWord[idxTemp] =', DexEntryWordByWord[idxTemp]) if debug else _
                    syllablesTemp = DexEntryWordByWord[idxTemp] # testing
                    print('syllablesTemp =', syllablesTemp) if debug else _
                print('type(syllablesTemp) =', type(syllablesTemp)) if debug else _
                syllablesTemp = syllablesTemp.split('-')
                
                print('syllablesTemp =', syllablesTemp) if debug else _
                if len(syllablesTemp)>1:
                    # try add one syllable + space + hyphene at the time
                    for syllableIdx in range(len(syllablesTemp)):
                        syllable = syllablesTemp[syllableIdx]
                        # first syllable is different
                        if syllableIdx==0:
                            stringTempNext = stringTemp + ' ' + syllable + '-'
                        else:
                            stringTempNext = stringTemp[:-1] + syllable + '-'
                        print('stringTempNext =', stringTempNext) if debug else _
                        # check if we went beyond the max allowed length
                        if len(stringTempNext)>lengthMax:
                            keepAddingWords = False
                            break
                        else:
                            # we succeded in adding a syllable to the line
                            stringTemp = stringTempNext
                            syllableIdx+=1
                    #break
                else:
                    # it can't be broken into syllables, henceforth (BEEEH) no hope, end of this line
                    break # keepAddingWords = False ???
        else:
            # all good, let's update the cycle
            stringTemp = stringTempNext
            idxTemp+=1
            # let's check if we ran out of words
            if idxTemp>=len(DexEntryWordByWord):
                wordsLeft = [] # mega ugly but works, a la GF
                break

        try:
            syllableIdx
        except:
            wordsLeft = DexEntryWordByWord[idxTemp:]
        else:
            if syllableIdx==0:
                wordsLeft = DexEntryWordByWord[idxTemp:]
            else:
                syllablesTempToBeGlued = syllablesTemp[syllableIdx:]
                leftoverWord = ''
                for wordPiece in syllablesTempToBeGlued:
                    leftoverWord += wordPiece
                wordsLeft = [leftoverWord] + DexEntryWordByWord[idxTemp+1:]
            print('wordsLeft =', wordsLeft) if debug else _
                
        print('stringTemp =', stringTemp) if debug else _

    return stringTemp, wordsLeft

def lineifyDexEntries(DexEntry='Hello world', lengthMax=18, numberOfLinesMax=6, replacePokemonShort=True, debug=False):
    DexEntry=DexEntry.replace('#MON', 'PoKeMoN') if replacePokemonShort else _
    
    if len(DexEntry)>lengthMax*numberOfLinesMax:
        print('You will not be able to fit your Pokedex entry!')
        print(f'Your Dex entry is {len(DexEntry)} characters long, but with the current settings')
        print(f'you can have max {numberOfLinesMax} lines, each {lengthMax} characters long, for a total of {lengthMax*numberOfLinesMax} characters.')
        print('The constraints above can be changed my inputting different "numberOfLinesMax" and "lengthMax" parameters.')
    else:
        DexEntryWordByWord = DexEntry.rsplit()
        DexEntry_Lineified = []
        while len(DexEntryWordByWord)>0:
            lineHere, DexEntryWordByWord = lineifyDexEntries_OneLine(DexEntryWordByWord, lengthMax=lengthMax, debug=debug)
            print('lineHere =', lineHere) if debug else _
            print('DexEntryWordByWord =', DexEntryWordByWord) if debug else _
            print('-----') if debug else _
            lineHere=lineHere.replace('PoKeMoN', '#MON') if replacePokemonShort else _
            DexEntry_Lineified.append(lineHere)
            
        if len(DexEntry_Lineified)>numberOfLinesMax:
            print('Sorry, I could not split the Dex entry into the required number of lines.')
            print('It may be possible to do so by changing (even against grammar rules) the hyphenation.')
            print('Here is the best I could do:')
            
        return DexEntry_Lineified


In [21]:
def lineifyDialogue(DexEntry='Hello world', lengthMax1=18, lengthMax2=17, replacePokemonShort=True, debug=False, firstLine=False, syllableBreaking=False):
    #DexEntry=DexEntry.replace('#MON', 'PoKeMoN') if replacePokemonShort else _
    #DexEntry=DexEntry.replace('<PLAYER>', '1234567') if replacePokemonShort else _

    DexEntryWordByWord = DexEntry.rsplit()
    #DexEntry_Lineified = []
    contLine = 0
    while len(DexEntryWordByWord)>0:
        lengthMaxHere=lengthMax1 if contLine==0 else lengthMax2
        lineHere, DexEntryWordByWord = lineifyDexEntries_OneLine(DexEntryWordByWord, lengthMax=lengthMaxHere, debug=debug, syllableBreaking=syllableBreaking)
        if debug:
            print('lineHere =', lineHere)
            print('DexEntryWordByWord =', DexEntryWordByWord)
            print('-----')
        lineHere=lineHere.replace('2345678', '<RIVAL>') if replacePokemonShort else _
        lineHere=lineHere.replace('1234567', '<PLAYER>') if replacePokemonShort else _
        lineHere=lineHere.replace('PoKeMoN', '#MON') if replacePokemonShort else _
        #DexEntry_Lineified.append(lineHere)
        lineBeginner = 'para "' if not firstLine else 'text "'
        if contLine==1:
            lineBeginner = 'line "'
        elif contLine>=2:
            lineBeginner = 'cont "'
        print(lineBeginner + lineHere + '"')
        
        contLine+=1
    #return DexEntry_Lineified


In [22]:
def getFileFolder(filein):
    filePath = ''
    for pathPiece in filein[:-len('.asm')].rsplit(r'/')[:-1]:
        filePath += pathPiece+r'/'
    return filePath

In [23]:
def tidyOldAndFromExcelFiles(fullNameOfFilesThatNeedsToBe, debug=False):
    oldFileToMove = fullNameOfFilesThatNeedsToBe
#    oldFileToMove = '/Users/Michael/Desktop/Core/Programmi_miei/Pokemon/pretStuff/outputPokemonInfoToExcel/Bands.asm'
    oldFileToMove_newName = oldFileToMove.replace(".asm", "_preExcel.asm")
    print('oldFileToMove =', oldFileToMove) if debug else _
    print('oldFileToMove_newName =', oldFileToMove_newName) if debug else _
    print('getFileFolder(oldFileToMove) =', getFileFolder(oldFileToMove)) if debug else _
    
    newFileToRename = oldFileToMove
    newFileToRename = newFileToRename.replace(".asm", "_fromExcel.asm")
    newFileToRename_newName = newFileToRename.replace("_fromExcel.asm", ".asm")
    print('newFileToRename =', newFileToRename) if debug else _
    print('newFileToRename_newName =', newFileToRename_newName) if debug else _

    #os.system('pwd') if debug else _
    # move in the right folder
    os.chdir(getFileFolder(oldFileToMove))
    os.system('pwd') if debug else _

    # make the dir for the old stuff
    command = 'mkdir preExcel'
    print(command) if debug else _
    os.system(command)
    time.sleep(1) if debug else _

    # rename the old files with a suffix (_preExcel)
    command = f'mv {oldFileToMove} {oldFileToMove_newName}'
    print(command) if debug else _
    os.system(command)
    time.sleep(1) if debug else _

    # move said renamed old files to the dir for old stuff
    command = f'mv {oldFileToMove_newName} ./preExcel'
    print(command) if debug else _
    os.system(command)
    time.sleep(1) if debug else _

    # rename the new files (_fromExcel, made from the script) by getting rid of their extension
    command = f'mv {newFileToRename} {newFileToRename_newName}'
    print(command) if debug else _
    os.system(command)


# Read mega file of destiny

In [12]:
#DfPkmnInfoFromExcel = pd.read_excel('/Users/Michael/Desktop/Core/Programmi_miei/Pokemon/pretStuff/outputPokemonInfoToExcel/testPokemonToExcel.ods', engine='odf')
DfPkmnInfoFromExcel = pd.read_excel('/Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/AllPokemonInfoExcel/PokemonToExcel.ods', sheet_name='Sheet1', engine='odf')

In [116]:
DfPkmnInfoFromExcel.head()

Unnamed: 0 Name_Capital Name_allsmall Name_PartlyCapital  HPs  ATK  DEF  \
0           1    BULBASAUR     bulbasaur          Bulbasaur   45   49   49   
1           2      IVYSAUR       ivysaur            Ivysaur   60   62   63   
2           3     VENUSAUR      venusaur           Venusaur   80   82   83   
3           4    MVENUSAUR     mvenusaur          MVenusaur   80  100  123   
4           5   CHARMANDER    charmander         Charmander   39   52   43   

   SPC  SPD  Type1   Type2  CatchRate  BaseExp  \
0   65   45  GRASS  POISON         45       64   
1   80   60  GRASS  POISON         45      141   
2  100   80  GRASS  POISON         45      208   
3  122   80  GRASS  POISON         45      233   
4   50   65   FIRE    FIRE         45       65   

                                 Lvl1Moves          GrowthRate  \
0          TACKLE, GROWL, NO_MOVE, NO_MOVE  GROWTH_MEDIUM_SLOW   
1       TACKLE, GROWL, LEECH_SEED, NO_MOVE  GROWTH_MEDIUM_SLOW   
2  BODY_SLAM, GROWL, LEECH_SEED, VINE_WHIP  GROWTH_MEDIUM_SLOW   
3     TACKLE, GROWL, LEECH_SEED, VINE_WHIP  GROWTH_MEDIUM_SLOW   
4         SCRATCH, GROWL, NO_MOVE, NO_MOVE  GROWTH_MEDIUM_SLOW   

                                           TMHMMoves  \
0  SWORDS_DANCE, TOXIC, BODY_SLAM, DOUBLE_EDGE, M...   
1  SWORDS_DANCE, TOXIC, BODY_SLAM, DOUBLE_EDGE, M...   
2  SWORDS_DANCE, TOXIC, BODY_SLAM, DOUBLE_EDGE, H...   
3  SWORDS_DANCE, TOXIC, BODY_SLAM, DOUBLE_EDGE, H...   
4  MEGA_PUNCH, SWORDS_DANCE, DRAGON_CLAW, TOXIC, ...   

                                             EvoInfo  \
0                              EV_LEVEL, 16, IVYSAUR   
1                             EV_LEVEL, 32, VENUSAUR   
2  EV_ITEM, MEGA_STONE_X, 1, MVENUSAUR ; EV_ITEM,...   
3                                                NaN   
4                           EV_LEVEL, 16, CHARMELEON   

                                        LevelUpMoves                 cries  \
0  7 LEECH_SEED ; 9 VINE_WHIP ; 13 POISONPOWDER ;...  SFX_CRY_0F, $80, $01   
1  7 LEECH_SEED ; 9 VINE_WHIP ; 13 POISONPOWDER ;...  SFX_CRY_0F, $20, $80   
2  7 LEECH_SEED ; 9 VINE_WHIP ; 13 POISONPOWDER ;...  SFX_CRY_0F, $00, $C0   
3  12 LEECH_SEED ; 14 VINE_WHIP ; 18 POISONPOWDER...  SFX_CRY_0F, $00, $C0   
4  9 EMBER ; 12 DRAGONBREATH ; 15 LEER ; 17 FIRE_...  SFX_CRY_04, $60, $40   

    menuIcons      palettes species  height_dm  weight_hg  \
0  ICON_GRASS  PAL_GREENMON    SEED          7         68   
1  ICON_GRASS  PAL_GREENMON    SEED         10        132   
2  ICON_GRASS  PAL_GREENMON    SEED         20       1002   
3  ICON_GRASS  PAL_GREENMON    SEED         24       1555   
4    ICON_MON    PAL_REDMON  LIZARD          6         86   

                                            dexTexts  Unnamed: 25  \
0  It can go for days without eating a single mor...          NaN   
1  The bulb on its back grows by drawing energy. ...          NaN   
2  The flower on its back catches the sun's rays....          NaN   
3  To support its flower, which is larger due to ...          NaN   
4  The flame at the tip of its tail makes a sound...          NaN   

   Unnamed: 26  Unnamed: 27  Unnamed: 28  Unnamed: 29  Unnamed: 30 ExtraNotes  \
0          NaN          NaN          NaN          NaN          NaN        NaN   
1          NaN          NaN          NaN          NaN          NaN        NaN   
2          NaN          NaN          NaN          NaN          NaN        NaN   
3          NaN          NaN          NaN          NaN          NaN        NaN   
4          NaN          NaN          NaN          NaN          NaN        NaN   

  Unnamed: 32  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN

In [23]:
allColumns_list = ['Name_Capital', 'Name_allsmall', 'Name_PartlyCapital', 'HPs', 'ATK', 'DEF', 'SPC', 'SPD', 'Type1', 'Type2', 'CatchRate', 'BaseExp', 'Lvl1Moves', 'GrowthRate', 'TMHMMoves', 'EvoInfo', 'LevelUpMoves', 'cries', 'menuIcons', 'palettes', 'species', 'height_dm', 'weight_hg', 'dexTexts', 'ExtraNotes']

In [26]:
for info in allColumns_list:
    print(f'{info}{printNCharacters(20-len(info))}= {printOneInfoForOneMon(info, "magikarp")}')
    

Name_Capital        = MAGIKARP
Name_allsmall       = magikarp
Name_PartlyCapital  = Magikarp
HPs                 = 20
ATK                 = 10
DEF                 = 55
SPC                 = 20
SPD                 = 80
Type1               = WATER
Type2               = WATER
CatchRate           = 255
BaseExp             = 20
Lvl1Moves           = SPLASH, NO_MOVE, NO_MOVE, NO_MOVE
GrowthRate          = GROWTH_SLOW
TMHMMoves           = FLAIL
EvoInfo             = EV_LEVEL, 20, GYARADOS
LevelUpMoves        = 10 TACKLE ; 20 FLAIL ; 50 DRAGON_RAGE
cries               = SFX_CRY_17, $80, $00
menuIcons           = ICON_WATER
palettes            = PAL_REDMON
species             = FISH
height_dm           = 9
weight_hg           = 100
dexTexts            = Famous for being very unreliable. It can be found swimming in seas, lakes, rivers and shallow puddles
ExtraNotes          = nan


In [119]:
printOneInfoForOneMon('LevelUpMoves', 'mrayquaza').rsplit(' ; ')

['9 CRUNCH',
 '12 SWIFT',
 '15 MUD_SLAP',
 '18 DRAGON_DANCE',
 '21 WRAP',
 '24 ROCK_TOMB',
 '27 EXTREMESPEED',
 '31 WATER_PULSE',
 '36 AERIAL_ACE',
 '40 SHOCK_WAVE',
 '44 HEADBUTT',
 '47 SHADOW_CLAW',
 '50 WATERFALL',
 '54 REST',
 '58 IRON_TAIL',
 '63 FLY',
 '67 ENERGY_BALL',
 '72 HURRICANE',
 '76 SOLARBEAM',
 '81 OUTRAGE',
 '85 FLAMETHROWER',
 '90 HYPER_BEAM',
 '95 DRAGONASCENT']

# preliminary check: are mons' names <= 10 chars? and are names consistent?

In [120]:
nameColumns_list = ['Name_Capital', 'Name_allsmall', 'Name_PartlyCapital', ]

In [121]:
DfPkmnInfoFromExcel_Name_Capital = DfPkmnInfoFromExcel['Name_Capital'].tolist()
DfPkmnInfoFromExcel_Name_allsmall = DfPkmnInfoFromExcel['Name_allsmall'].tolist()
DfPkmnInfoFromExcel_Name_PartlyCapital = DfPkmnInfoFromExcel['Name_PartlyCapital'].tolist()

In [122]:
for idx in range(len(DfPkmnInfoFromExcel)):
    if not ((DfPkmnInfoFromExcel_Name_Capital[idx]==DfPkmnInfoFromExcel_Name_allsmall[idx].upper()) and (DfPkmnInfoFromExcel_Name_Capital[idx]==DfPkmnInfoFromExcel_Name_PartlyCapital[idx].upper())):
        print('Names not equal!')
        print(DfPkmnInfoFromExcel_Name_Capital[idx])
        print(DfPkmnInfoFromExcel_Name_allsmall[idx])
        print(DfPkmnInfoFromExcel_Name_PartlyCapital[idx])
    if len(DfPkmnInfoFromExcel_Name_Capital[idx])>10:
        print('Len > 10!')
    

Names not equal!
NIDORAN_F
nidoranf
NidoranF
Names not equal!
NIDORAN_M
nidoranm
NidoranM
Names not equal!
MIME_JR
mimejr
MimeJr
Names not equal!
MR_MIME
mrmime
MrMime


# pokemon_constants (1/13)

In [123]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/constants/pokemon_constants.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [124]:
initialLines_firstIdx = 0
initialLines_lastIdx = 8

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [125]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-12

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [126]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')+listFossilsGhost_Capital
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tconst {pkmn}{printNCharacters(19-len(pkmn))}; {idx+1}\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [127]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# pokedex_constants (2/13)

In [128]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/constants/pokedex_constants.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [129]:
initialLines_firstIdx = 0
initialLines_lastIdx = 5

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [130]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-1

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [131]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tconst DEX_{pkmn}{printNCharacters(11-len(pkmn))}; {idx+1}\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [132]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# base_stats single files (3/13)

In [27]:
listMonsHere_allsmall = getOneColumn('Name_allsmall')
listMonsHere_Capital = getOneColumn('Name_Capital')
listMonsHere_PartlyCapital = getOneColumn('Name_PartlyCapital')
for pkmnidx in range(len(listMonsHere_allsmall)):
    pkmnHere_allsmall = listMonsHere_allsmall[pkmnidx]
    pkmnHere_Capital = listMonsHere_Capital[pkmnidx]
    pkmnHere_PartlyCapital = listMonsHere_PartlyCapital[pkmnidx]
    
    inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/base_stats/'+pkmnHere_allsmall+'.asm'
    linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
    verbose = True

    txtToWrite = open(outputFile, "w")

    txtToWrite.write(f'\tdb DEX_{pkmnHere_Capital} ; pokedex id\n')
    txtToWrite.write(f'\n')

    lineToWrite = '\tdb'
    for statistic in ['HPs', 'ATK', 'DEF', 'SPD', 'SPC']:
        lineToWrite = lineToWrite + printNCharacters(4-len(str(printOneInfoForOneMon(statistic,pkmnHere_allsmall)))) + str(printOneInfoForOneMon(statistic,pkmnHere_allsmall)) + ','
    lineToWrite = lineToWrite[:-1] + '\n' 
    txtToWrite.write(lineToWrite)

    txtToWrite.write(f'\t;   hp  atk  def  spd  spc\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("Type1",pkmnHere_allsmall)}, {printOneInfoForOneMon("Type2",pkmnHere_allsmall)} ; type\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("CatchRate",pkmnHere_allsmall)} ; catch rate\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("BaseExp",pkmnHere_allsmall)} ; base exp\n')
    txtToWrite.write(f'\n')
    txtToWrite.write('\tINCBIN "gfx/pokemon/front/'+pkmnHere_allsmall+'.pic", 0, 1 ; sprite dimensions\n')
    txtToWrite.write(f'\tdw {pkmnHere_PartlyCapital}PicFront, {pkmnHere_PartlyCapital}PicBack\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("Lvl1Moves",pkmnHere_allsmall)} ; level 1 learnset\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("GrowthRate",pkmnHere_allsmall)} ; growth rate\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\t; tm/hm learnset\n')

    try:
        len(printOneInfoForOneMon("TMHMMoves",pkmnHere_allsmall))
    except:
        txtToWrite.write('\ttmhm\n')
    else:
        THMLeft = printOneInfoForOneMon("TMHMMoves",pkmnHere_allsmall).rsplit()
        while len(THMLeft)>0:
            lineToWrite='\ttmhm ' if len(THMLeft)==len(printOneInfoForOneMon("TMHMMoves",pkmnHere_allsmall).rsplit()) else '\t     '
            lineEnd='\\\n' if len(THMLeft)>5 else '\n'
            for idxTHM in range(min(len(THMLeft),5)):
                lineToWrite = lineToWrite + THMLeft[0] + printNCharacters(14-len(THMLeft[0]))
                THMLeft.pop(0)
            lineToWrite += lineEnd
            txtToWrite.write(lineToWrite)

    txtToWrite.write(f'\t; end\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\tdb BANK({pkmnHere_PartlyCapital}PicFront)\n')
    txtToWrite.write(f'\tassert BANK({pkmnHere_PartlyCapital}PicFront) == BANK({pkmnHere_PartlyCapital}PicBack)\n')

    txtToWrite.close()
    
    tidyOldAndFromExcelFiles(inputFile, debug=False)
    

mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExcel: File exists
mkdir: preExce

# base_stats "folder" (4/13)

In [28]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/base_stats.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [135]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [136]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [137]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_allsmall')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = 'INCLUDE "data/pokemon/base_stats/'+pkmn+'.asm"\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [138]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# evos_moves (5/13)

In [39]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/evos_moves.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [40]:
initialLines_firstIdx = 0
initialLines_lastIdx = 4

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [41]:
middleLines = ['; map pieces, new\n'] + np.full(15, '\tdw MapPiecesEvosMoves\n').tolist() + ['\tassert_table_length NUM_POKEMON_INDEXES\n', '\n']

In [42]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tdw {pkmn}EvosMoves\n'
    txtToWrite.write(oneLine)
for crapMon in listFossilsGhost:
    txtToWrite.write(f'\tdw {crapMon}EvosMoves\n')
    
for oneLine in middleLines:
    txtToWrite.write(oneLine)

for pkmn in listMonsHere:   
    txtToWrite.write(f'{pkmn}EvosMoves:\n')
    txtToWrite.write(f'; Evolutions\n')
    
    try:
        len(printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital'))
    except:
        _
    else:
#        if not ' - ' in printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital'):
#            txtToWrite.write(f'\tdb {printOneInfoForOneMon("EvoInfo", pkmn, "Name_PartlyCapital")}\n')
#        else:
#            for evolutionOption in printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital').rsplit(' - '):
#                txtToWrite.write(f'\tdb {evolutionOption}\n')
        for evolutionOption in printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital').rsplit(' ; '):
            txtToWrite.write(f'\tdb {evolutionOption}\n')
    
    txtToWrite.write(f'\tdb 0\n')
    txtToWrite.write(f'; Learnset\n')
    
    try:
        len(printOneInfoForOneMon('LevelUpMoves', pkmn, 'Name_PartlyCapital'))
    except:
        _
    else:
        for lvlUpMove in printOneInfoForOneMon('LevelUpMoves', pkmn, 'Name_PartlyCapital').rsplit(' ; '):
            txtToWrite.write(f'\tdb {lvlUpMove.rsplit()[0]}, {lvlUpMove.rsplit()[1]}\n')
    
    txtToWrite.write(f'\tdb 0\n')
    txtToWrite.write(f'\n')
    
txtToWrite.write(f'MissingNoEvosMoves:\n')
txtToWrite.write(f'; Evolutions\n') 
txtToWrite.write(f'\tdb 0\n')
txtToWrite.write(f'; Learnset\n')
txtToWrite.write(f'\tdb 0\n')
txtToWrite.write(f'\n')

txtToWrite.write(f'; rearranged to have all the proxies together\n')
for crapMon in listFossilsGhost:
    txtToWrite.write(f'{crapMon}EvosMoves:\n')
    
txtToWrite.write(f'; Evolutions\n') 
txtToWrite.write(f'\tdb 0\n')
txtToWrite.write(f'; Learnset\n')
txtToWrite.write(f'\tdb 0\n')
txtToWrite.write(f'\n')
    
txtToWrite.close()

In [43]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# names (6/13)

In [144]:
names_Exceptions_Default = ["NIDORAN_M", "NIDORAN_F", "MIME_JR", "MR_MIME", "FARFETCHD", "MISSINGNO", "BLUESDAD", "BLUESMOM"]
names_Exceptions_Updated = ["NIDORAN♂", "NIDORAN♀", "MIME JR.", "MR.MIME", "FARFETCH'D", "MISSINGNO.", "BLUE's DAD", "BLUE's MOM"]

In [145]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/names.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [146]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [147]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [148]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    if pkmn in names_Exceptions_Default:
        pkmn = names_Exceptions_Updated[names_Exceptions_Default.index(pkmn)]
    oneLine = '\tdb "'+pkmn+printNCharacters(10-len(pkmn), '@')+'"\n'
    txtToWrite.write(oneLine)

for crapMon in listFossilsGhost_Names_Capital:
    oneLine = '\tdb "'+crapMon+printNCharacters(10-len(crapMon), '@')+'"\n'
    txtToWrite.write(oneLine)
    
for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [149]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# cries (7/13)

In [150]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/cries.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [151]:
initialLines_firstIdx = 0
initialLines_lastIdx = 7

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [152]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [153]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tmon_cry {printOneInfoForOneMon("cries", pkmn, "Name_PartlyCapital")} ; {pkmn}\n'
    txtToWrite.write(oneLine)

for crapMon in listFossilsGhost_Names:
    oneLine = f'\tmon_cry SFX_CRY_00, $00, $00 ; {crapMon}\n'
    txtToWrite.write(oneLine)
    
for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [154]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# menu_icons (8/13)

In [155]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/menu_icons.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [156]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [157]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [158]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = '\tnybble '+printOneInfoForOneMon("menuIcons", pkmn, "Name_PartlyCapital")+printNCharacters(15-len(printOneInfoForOneMon("menuIcons", pkmn, "Name_PartlyCapital")))+'; '+pkmn+'\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [159]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# palettes (9/13)

In [160]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/palettes.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [161]:
initialLines_firstIdx = 0
initialLines_lastIdx = 2

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [162]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [163]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = '\tdb '+printOneInfoForOneMon("palettes", pkmn, "Name_Capital")+printNCharacters(14-len(printOneInfoForOneMon("palettes", pkmn, "Name_Capital")))+'; '+pkmn+'\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [164]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# dex_order (10/13)

In [165]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_order.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [166]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [167]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [168]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tdb DEX_{pkmn}\n'
    txtToWrite.write(oneLine)

for crapMon in listFossilsGhost_Names_Capital:
    oneLine = f'\tdb 0 ; {crapMon}\n'
    txtToWrite.write(oneLine)
    
for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [169]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# dex_entries (11/13)

In [170]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_entries.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [171]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [172]:
middleLines = ['\tassert_table_length NUM_POKEMON_INDEXES\n', '\n', '; string: species name\n', '; height in decimeter\n', '; weight in hectograms\n', '; text entry\n', '\n', ]

In [173]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tdw {pkmn}DexEntry\n'
    txtToWrite.write(oneLine)
for crapMon in listFossilsGhost_Names:
    txtToWrite.write(f'\tdw {crapMon[:-1]}DexEntry\n')
    
for oneLine in middleLines:
    txtToWrite.write(oneLine)

for pkmn in listMonsHere:   
    txtToWrite.write(f'{pkmn}DexEntry:\n')
    txtToWrite.write('\tdb "'+printOneInfoForOneMon('species', pkmn, 'Name_PartlyCapital')+'@"\n')
    txtToWrite.write('\tdb '+str(printOneInfoForOneMon('height_dm', pkmn, 'Name_PartlyCapital'))+'\n')
    txtToWrite.write('\tdw '+str(printOneInfoForOneMon('weight_hg', pkmn, 'Name_PartlyCapital'))+'\n')
    txtToWrite.write(f'\ttext_far _{pkmn}DexEntry\n')
    txtToWrite.write('\ttext_end\n')
    txtToWrite.write('\n')
    
txtToWrite.write('MissingNoDexEntry:\n')
txtToWrite.write('\tdb "???@"\n') 
txtToWrite.write('\tdb 31\n') 
txtToWrite.write('\tdw 45\n') 
txtToWrite.write('\tdb "コメント\u3000さくせいちゅう@" ; コメント作成中 (Comment to be written)\n') 

txtToWrite.close()

In [174]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# dex_text (12/13)

In [175]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_text.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [176]:
# cross-checks
# - are all entries theoretically fittable?
for pkmn in getOneColumn('Name_PartlyCapital'):
    if len(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))>18*6:
        print(f'{pkmn} has a too-long entry!')
# - are all entries re-made in 6 lines?
for pkmn in getOneColumn('Name_PartlyCapital'):
    if len(lineifyDexEntries(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital')))!=6:
        print(f'{pkmn} has a '+str(len(lineifyDexEntries(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))))+'-line entry!')

Squirtle has a 5-line entry!
Raticate has a 5-line entry!
Gengar has a 5-line entry!
Magmortar has a 5-line entry!
Sylveon has a 4-line entry!
MissingNo has a 2-line entry!
BluesDad has a 5-line entry!
BluesMom has a 5-line entry!
Dad has a 5-line entry!


In [177]:
txtToWrite = open(outputFile, "w")

listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
#    print(pkmn)
#    print(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))
    dexLinesHere = lineifyDexEntries(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))
#    print(dexLinesHere)
    txtToWrite.write(f'_{pkmn}DexEntry::\n')
    if len(dexLinesHere)>=1:
        txtToWrite.write('\ttext "'+dexLinesHere[0]+'"\n')
    if len(dexLinesHere)>=2:
        txtToWrite.write('\tnext "'+dexLinesHere[1]+'"\n')
    if len(dexLinesHere)>=3:
        txtToWrite.write('\tnext "'+dexLinesHere[2]+'"\n')
        txtToWrite.write('\n')
    if len(dexLinesHere)>=4:
        txtToWrite.write('\tpage "'+dexLinesHere[3]+'"\n')
    if len(dexLinesHere)>=5:
        txtToWrite.write('\tnext "'+dexLinesHere[4]+'"\n')
    if len(dexLinesHere)==6:
        txtToWrite.write('\tnext "'+dexLinesHere[5]+'"\n')
    txtToWrite.write('\tdex\n')
    txtToWrite.write('\n')
#    print('---')
    
txtToWrite.close()

In [178]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# pics D: (13/13)

In [333]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pics.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [334]:
initialLines_firstIdx = linesFromFile.index('SECTION "Pics 6", ROMX\n')
initialLines_lastIdx = linesFromFile.index('SECTION "Pics 8", ROMX\n')+7

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [335]:
txtToWrite = open(outputFile, "w")

listMonsHere = getOneColumn('Name_PartlyCapital')
listMonsHere_allsmall = getOneColumn('Name_allsmall')

for idx in range(len(listMonsHere)):
    
    if int(idx/30)+1==6:
        txtToWrite.write('\n')
        txtToWrite.write('\n')
#        txtToWrite.write('SECTION "Pics '+str(int(idx/30)+2)+'", ROMX\n')
        txtToWrite.write('SECTION "Pics '+str(6)+'", ROMX\n')
        txtToWrite.write('\n')
        txtToWrite.write('YoungsterPic::     INCBIN "gfx/trainers/youngster.pic"\n')
        txtToWrite.write('BugCatcherPic::    INCBIN "gfx/trainers/bugcatcher.pic"\n')
        txtToWrite.write('LassPic::          INCBIN "gfx/trainers/lass.pic"\n')
        txtToWrite.write('SailorPic::        INCBIN "gfx/trainers/sailor.pic"\n')
        txtToWrite.write('JrTrainerMPic::    INCBIN "gfx/trainers/jr.trainerm.pic"\n')
        txtToWrite.write('JrTrainerFPic::    INCBIN "gfx/trainers/jr.trainerf.pic"\n')
        txtToWrite.write('PokemaniacPic::    INCBIN "gfx/trainers/pokemaniac.pic"\n')
        txtToWrite.write('SuperNerdPic::     INCBIN "gfx/trainers/supernerd.pic"\n')
        txtToWrite.write('HikerPic::         INCBIN "gfx/trainers/hiker.pic"\n')
        txtToWrite.write('BikerPic::         INCBIN "gfx/trainers/biker.pic"\n')
        txtToWrite.write('BurglarPic::       INCBIN "gfx/trainers/burglar.pic"\n')
        txtToWrite.write('EngineerPic::      INCBIN "gfx/trainers/engineer.pic"\n')
        txtToWrite.write('FisherPic::        INCBIN "gfx/trainers/fisher.pic"\n')
        txtToWrite.write('SwimmerPic::       INCBIN "gfx/trainers/swimmer.pic"\n')
        txtToWrite.write('CueBallPic::       INCBIN "gfx/trainers/cueball.pic"\n')
        txtToWrite.write('GamblerPic::       INCBIN "gfx/trainers/gambler.pic"\n')
        txtToWrite.write('BeautyPic::        INCBIN "gfx/trainers/beauty.pic"\n')
        txtToWrite.write('PsychicPic::       INCBIN "gfx/trainers/psychic.pic"\n')
        txtToWrite.write('RockerPic::        INCBIN "gfx/trainers/rocker.pic"\n')
        txtToWrite.write('JugglerPic::       INCBIN "gfx/trainers/juggler.pic"\n')
        txtToWrite.write('TamerPic::         INCBIN "gfx/trainers/tamer.pic"\n')
        txtToWrite.write('BirdKeeperPic::    INCBIN "gfx/trainers/birdkeeper.pic"\n')
        txtToWrite.write('BlackbeltPic::     INCBIN "gfx/trainers/blackbelt.pic"\n')
        txtToWrite.write('Rival1Pic::        INCBIN "gfx/trainers/rival1.pic"\n')
        txtToWrite.write('ProfOakPic::       INCBIN "gfx/trainers/prof.oak.pic"\n')
        txtToWrite.write('ChiefPic::\n')
        txtToWrite.write('ScientistPic::     INCBIN "gfx/trainers/scientist.pic"\n')
        txtToWrite.write('GiovanniPic::      INCBIN "gfx/trainers/giovanni.pic"\n')
        txtToWrite.write('RocketPic::        INCBIN "gfx/trainers/rocket.pic"\n')
        txtToWrite.write('CooltrainerMPic::  INCBIN "gfx/trainers/cooltrainerm.pic"\n')
        txtToWrite.write('CooltrainerFPic::  INCBIN "gfx/trainers/cooltrainerf.pic"\n')
        txtToWrite.write('BrunoPic::         INCBIN "gfx/trainers/bruno.pic"\n')
        txtToWrite.write('BrockPic::         INCBIN "gfx/trainers/brock.pic"\n')
        txtToWrite.write('MistyPic::         INCBIN "gfx/trainers/misty.pic"\n')
        txtToWrite.write('LtSurgePic::       INCBIN "gfx/trainers/lt.surge.pic"\n')
        txtToWrite.write('ErikaPic::         INCBIN "gfx/trainers/erika.pic"\n')
        txtToWrite.write('KogaPic::          INCBIN "gfx/trainers/koga.pic"\n')
        txtToWrite.write('BlainePic::        INCBIN "gfx/trainers/blaine.pic"\n')
        txtToWrite.write('SabrinaPic::       INCBIN "gfx/trainers/sabrina.pic"\n')
        txtToWrite.write('GentlemanPic::     INCBIN "gfx/trainers/gentleman.pic"\n')
        txtToWrite.write('Rival2Pic::        INCBIN "gfx/trainers/rival2.pic"\n')
        txtToWrite.write('Rival3Pic::        INCBIN "gfx/trainers/rival3.pic"\n')
        txtToWrite.write('LoreleiPic::       INCBIN "gfx/trainers/lorelei.pic"\n')
        txtToWrite.write('ChannelerPic::     INCBIN "gfx/trainers/channeler.pic"\n')
        txtToWrite.write('AgathaPic::        INCBIN "gfx/trainers/agatha.pic"\n')
        txtToWrite.write('LancePic::         INCBIN "gfx/trainers/lance.pic"\n')
        txtToWrite.write('JessieJamesPic::   INCBIN "gfx/trainers/jessiejames.pic"\n')
        txtToWrite.write('\n')
        txtToWrite.write('\n')
#        txtToWrite.write('SECTION "Pics '+str(int(idx/30)+3)+'", ROMX\n')
        txtToWrite.write('SECTION "Pics '+str(7)+'", ROMX\n')
        txtToWrite.write('\n')
        txtToWrite.write('RedPicBack::       INCBIN "gfx/player/redb.pic"\n')
        txtToWrite.write('OldManPicBack::    INCBIN "gfx/battle/oldmanb.pic"\n')
        txtToWrite.write('ProfOakPicBack::   INCBIN "gfx/battle/prof.oakb.pic"\n')
        break
        
    if idx%30==0:
        if idx!=0:
            txtToWrite.write('\n')
            txtToWrite.write('\n')
        txtToWrite.write('SECTION "Pics '+str(int(idx/30)+1)+'", ROMX\n')
        txtToWrite.write('\n')
    pkmn = listMonsHere[idx]
    txtToWrite.write(pkmn+'PicFront::'+printNCharacters(22-len(pkmn+'PicFront::'))+'INCBIN "gfx/pokemon/front/'+listMonsHere_allsmall[idx]+'.pic"\n')
    txtToWrite.write(pkmn+'PicBack::'+printNCharacters(22-len(pkmn+'PicBack::'))+'INCBIN "gfx/pokemon/back/'+listMonsHere_allsmall[idx]+'b.pic"\n')
    
for idx2 in range(idx,len(listMonsHere)):
    if idx2%30==0:
        if idx2!=0:
            txtToWrite.write('\n')
            txtToWrite.write('\n')
        txtToWrite.write('SECTION "Pics '+str(int(idx2/30)+3)+'", ROMX\n')
        txtToWrite.write('\n')
    pkmn = listMonsHere[idx2]
    txtToWrite.write(pkmn+'PicFront::'+printNCharacters(22-len(pkmn+'PicFront::'))+'INCBIN "gfx/pokemon/front/'+listMonsHere_allsmall[idx2]+'.pic"\n')
    txtToWrite.write(pkmn+'PicBack::'+printNCharacters(22-len(pkmn+'PicBack::'))+'INCBIN "gfx/pokemon/back/'+listMonsHere_allsmall[idx2]+'b.pic"\n')
        
txtToWrite.write('\n')
txtToWrite.write('FossilKabutopsPic::   INCBIN "gfx/pokemon/front/fossilkabutops.pic"\n')
txtToWrite.write('FossilAerodactylPic::  INCBIN "gfx/pokemon/front/fossilaerodactyl.pic"\n')
txtToWrite.write('GhostPic::             INCBIN "gfx/battle/ghost.pic"\n')

txtToWrite.close()    

In [336]:
if False:
    txtToWrite = open(outputFile, "w")

    listMonsHere = getOneColumn('Name_PartlyCapital')
    listMonsHere_allsmall = getOneColumn('Name_allsmall')
    for idx in range(len(listMonsHere)):
        if idx%30==0:
            if idx!=0:
                txtToWrite.write('\n')
                txtToWrite.write('\n')
            txtToWrite.write('SECTION "Pics '+str(int(idx/30)+1)+'", ROMX\n')
            txtToWrite.write('\n')
        pkmn = listMonsHere[idx]
        txtToWrite.write(pkmn+'PicFront::'+printNCharacters(22-len(pkmn+'PicFront::'))+'INCBIN "gfx/pokemon/front/'+listMonsHere_allsmall[idx]+'.pic"\n')
        txtToWrite.write(pkmn+'PicBack::'+printNCharacters(22-len(pkmn+'PicBack::'))+'INCBIN "gfx/pokemon/back/'+listMonsHere_allsmall[idx]+'b.pic"\n')

    txtToWrite.write('\n')
    txtToWrite.write('\n')
    txtToWrite.write('SECTION "Pics '+str(int(idx/30)+2)+'", ROMX\n')
    txtToWrite.write('\n')
    txtToWrite.write('YoungsterPic::     INCBIN "gfx/trainers/youngster.pic"\n')
    txtToWrite.write('BugCatcherPic::    INCBIN "gfx/trainers/bugcatcher.pic"\n')
    txtToWrite.write('LassPic::          INCBIN "gfx/trainers/lass.pic"\n')
    txtToWrite.write('SailorPic::        INCBIN "gfx/trainers/sailor.pic"\n')
    txtToWrite.write('JrTrainerMPic::    INCBIN "gfx/trainers/jr.trainerm.pic"\n')
    txtToWrite.write('JrTrainerFPic::    INCBIN "gfx/trainers/jr.trainerf.pic"\n')
    txtToWrite.write('PokemaniacPic::    INCBIN "gfx/trainers/pokemaniac.pic"\n')
    txtToWrite.write('SuperNerdPic::     INCBIN "gfx/trainers/supernerd.pic"\n')
    txtToWrite.write('HikerPic::         INCBIN "gfx/trainers/hiker.pic"\n')
    txtToWrite.write('BikerPic::         INCBIN "gfx/trainers/biker.pic"\n')
    txtToWrite.write('BurglarPic::       INCBIN "gfx/trainers/burglar.pic"\n')
    txtToWrite.write('EngineerPic::      INCBIN "gfx/trainers/engineer.pic"\n')
    txtToWrite.write('FisherPic::        INCBIN "gfx/trainers/fisher.pic"\n')
    txtToWrite.write('SwimmerPic::       INCBIN "gfx/trainers/swimmer.pic"\n')
    txtToWrite.write('CueBallPic::       INCBIN "gfx/trainers/cueball.pic"\n')
    txtToWrite.write('GamblerPic::       INCBIN "gfx/trainers/gambler.pic"\n')
    txtToWrite.write('BeautyPic::        INCBIN "gfx/trainers/beauty.pic"\n')
    txtToWrite.write('PsychicPic::       INCBIN "gfx/trainers/psychic.pic"\n')
    txtToWrite.write('RockerPic::        INCBIN "gfx/trainers/rocker.pic"\n')
    txtToWrite.write('JugglerPic::       INCBIN "gfx/trainers/juggler.pic"\n')
    txtToWrite.write('TamerPic::         INCBIN "gfx/trainers/tamer.pic"\n')
    txtToWrite.write('BirdKeeperPic::    INCBIN "gfx/trainers/birdkeeper.pic"\n')
    txtToWrite.write('BlackbeltPic::     INCBIN "gfx/trainers/blackbelt.pic"\n')
    txtToWrite.write('Rival1Pic::        INCBIN "gfx/trainers/rival1.pic"\n')
    txtToWrite.write('ProfOakPic::       INCBIN "gfx/trainers/prof.oak.pic"\n')
    txtToWrite.write('ChiefPic::\n')
    txtToWrite.write('ScientistPic::     INCBIN "gfx/trainers/scientist.pic"\n')
    txtToWrite.write('GiovanniPic::      INCBIN "gfx/trainers/giovanni.pic"\n')
    txtToWrite.write('RocketPic::        INCBIN "gfx/trainers/rocket.pic"\n')
    txtToWrite.write('CooltrainerMPic::  INCBIN "gfx/trainers/cooltrainerm.pic"\n')
    txtToWrite.write('CooltrainerFPic::  INCBIN "gfx/trainers/cooltrainerf.pic"\n')
    txtToWrite.write('BrunoPic::         INCBIN "gfx/trainers/bruno.pic"\n')
    txtToWrite.write('BrockPic::         INCBIN "gfx/trainers/brock.pic"\n')
    txtToWrite.write('MistyPic::         INCBIN "gfx/trainers/misty.pic"\n')
    txtToWrite.write('LtSurgePic::       INCBIN "gfx/trainers/lt.surge.pic"\n')
    txtToWrite.write('ErikaPic::         INCBIN "gfx/trainers/erika.pic"\n')
    txtToWrite.write('KogaPic::          INCBIN "gfx/trainers/koga.pic"\n')
    txtToWrite.write('BlainePic::        INCBIN "gfx/trainers/blaine.pic"\n')
    txtToWrite.write('SabrinaPic::       INCBIN "gfx/trainers/sabrina.pic"\n')
    txtToWrite.write('GentlemanPic::     INCBIN "gfx/trainers/gentleman.pic"\n')
    txtToWrite.write('Rival2Pic::        INCBIN "gfx/trainers/rival2.pic"\n')
    txtToWrite.write('Rival3Pic::        INCBIN "gfx/trainers/rival3.pic"\n')
    txtToWrite.write('LoreleiPic::       INCBIN "gfx/trainers/lorelei.pic"\n')
    txtToWrite.write('ChannelerPic::     INCBIN "gfx/trainers/channeler.pic"\n')
    txtToWrite.write('AgathaPic::        INCBIN "gfx/trainers/agatha.pic"\n')
    txtToWrite.write('LancePic::         INCBIN "gfx/trainers/lance.pic"\n')
    txtToWrite.write('JessieJamesPic::   INCBIN "gfx/trainers/jessiejames.pic"\n')
    txtToWrite.write('\n')
    txtToWrite.write('\n')
    txtToWrite.write('SECTION "Pics '+str(int(idx/30)+3)+'", ROMX\n')
    txtToWrite.write('\n')
    txtToWrite.write('RedPicBack::       INCBIN "gfx/player/redb.pic"\n')
    txtToWrite.write('OldManPicBack::    INCBIN "gfx/battle/oldmanb.pic"\n')
    txtToWrite.write('ProfOakPicBack::   INCBIN "gfx/battle/prof.oakb.pic"\n')
    txtToWrite.write('\n')
    txtToWrite.write('\n')
    txtToWrite.write('SECTION "Pics '+str(int(idx/30)+4)+'", ROMX\n')
    txtToWrite.write('\n')
    txtToWrite.write('FossilKabutopsPic::   INCBIN "gfx/pokemon/front/fossilkabutops.pic"\n')
    txtToWrite.write('FossilAerodactylPic::  INCBIN "gfx/pokemon/front/fossilaerodactyl.pic"\n')
    txtToWrite.write('GhostPic::             INCBIN "gfx/battle/ghost.pic"\n')

    txtToWrite.close()    

In [337]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

mkdir: preExcel: File exists


# Proxy images just to test

In [14]:
pkmnWithProxyPics_list = ['mvenusaur', 'mcharzardx', 'mcharzardy', 'mblastoise', 'mbeedrill', 'mpidgeot', 'pichu', 'cleffa', \
                          'igglybuff', 'crobat', 'bellossom', 'annihilape', 'politoed', 'malakazam', 'mslowbro', 'slowking', \
                          'magnezone', 'mgengar', 'steelix', 'msteelix', 'tyrogue', 'hitmontop', 'lickilicky', 'rhyperior', \
                          'happiny', 'blissey', 'tangrowth', 'mkangaskan', 'kingdra', 'mimejr', 'scizor', 'mscizor', 'kleavor', \
                          'smoochum', 'elekid', 'electivire', 'magby', 'magmortar', 'mpinsir', 'mgyarados', 'espeon', 'umbreon', \
                          'leafeon', 'glaceon', 'sylveon', 'porygon2', 'porygonz', 'maerodatyl', 'munchlax', 'mmewtwox', 'mmewtwoy', \
                          'venustoise']

pkmnWithProxyPics_list = ['mvenusaur', 'mcharzardx', 'mcharzardy', 'mblastoise', 'mbeedrill', 'mpidgeot', \
                          'annihilape', \
                          'mgengar', 'msteelix', \
                          'mscizor', 'kleavor', \
                          'mpinsir', 'mgyarados', \
                          'maerodatyl', 'mmewtwox', 'mmewtwoy', ]
                                                    

In [15]:
for pkmnToProxy in pkmnWithProxyPics_list:
    print(pkmnToProxy)
    command = f'cp /Users/Michael/Downloads/Proxy_Back.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/back/{pkmnToProxy}b.png'
    print(command)
    os.system(command)
    command = f'cp /Users/Michael/Downloads/Proxy_Front.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/front/{pkmnToProxy}.png'
    print(command)
    os.system(command)
    print('---')

mvenusaur
cp /Users/Michael/Downloads/Proxy_Back.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/back/mvenusaurb.png
cp /Users/Michael/Downloads/Proxy_Front.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/front/mvenusaur.png
---
mcharzardx
cp /Users/Michael/Downloads/Proxy_Back.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/back/mcharzardxb.png
cp /Users/Michael/Downloads/Proxy_Front.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/front/mcharzardx.png
---
mcharzardy
cp /Users/Michael/Downloads/Proxy_Back.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/back/mcharzardyb.png
cp /Users/Michael/Downloads/Proxy_Front.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/front/mcharzardy.png
---
mblastoise
cp /Users/Michael/Downloads/Prox

In [15]:
pkmnWithNewAlreadyDonePics_list = ['pichu', 'cleffa', \
                                   'igglybuff', 'crobat', 'bellossom', 'politoed', 'slowking', \
                                   'magnezone', 'steelix', 'tyrogue', 'hitmontop', 'lickilicky', 'rhyperior', \
                                   'happiny', 'blissey', 'tangrowth', 'kingdra', 'mimejr', 'scizor', \
                                   'smoochum', 'elekid', 'electivire', 'magby', 'magmortar', 'espeon', 'umbreon', \
                                   'leafeon', 'glaceon', 'sylveon', 'porygon2', 'porygonz', 'munchlax',]
len(pkmnWithNewAlreadyDonePics_list)

32

In [22]:
for pkmnToFix in pkmnWithNewAlreadyDonePics_list:
#    print(pkmnToFix)
    command = f'python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/{pkmnToFix}.png'
    print(command)
#    os.system(command)
#    print('---')

python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/pichu.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/cleffa.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/igglybuff.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/crobat.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/bellossom.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/politoed.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/done/slowking.png
python /Users/Michael/Desktop/Co

In [ ]:
for pkmnToFix in pkmnWithNewAlreadyDonePics_list:
    print(pkmnToFix)
    command = f'python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/{pkmnToFix}.png'
    print(command)
    os.system(command)
    print('---')

In [ ]:
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/igglybuff.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/magnezone.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/rhyperior.png
python /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/tools/palfix.py gfx/pokemon/new/front/kingdra.png


# Sorted THMs

In [ ]:
THMs_sorted = ['MEGA_PUNCH', 'MEGAHORN', 'SWORDS_DANCE', 'COUNTER', 'DRAGON_CLAW', 'TOXIC', 'DARK_PULSE', 'BODY_SLAM', 'METEOR_MASH', 'DOUBLE_EDGE', 'BUBBLEBEAM', 'METRONOME', 'ICE_BEAM', 'BLIZZARD', 'HYPER_BEAM', 'MIRROR_COAT', 'DRAININGKISS', 'FLASH_CANNON', 'SEISMIC_TOSS', 'FEINT_ATTACK', 'GIGA_DRAIN', 'STONE_EDGE', 'WILL_O_WISP', 'THUNDERBOLT', 'THUNDER', 'EARTHQUAKE', 'FISSURE', 'DIG', 'PSYCHIC_M', 'MOONBLAST', 'MIMIC', 'DOUBLE_TEAM', 'REFLECT', 'BIDE', 'HYDRO_PUMP', 'SELFDESTRUCT', 'CLOSE_COMBAT', 'FIRE_BLAST', 'SHADOW_SNEAK', 'OUTRAGE', 'BRAVE_BIRD', 'REST', 'GUNK_SHOT', 'X_SCISSOR', 'THUNDER_WAVE', 'PSYWAVE', 'EXPLOSION', 'ROCK_SLIDE', 'SHADOW_BALL', 'SUBSTITUTE', ]

In [ ]:
len(THMs_sorted)

# Test length of dex entry

In [114]:
dexEntry_here = "It absorbed overwhelming light energy. It fires lasers that affect every living being"

lineifyDexEntries(dexEntry_here, numberOfLinesMax=6, lengthMax=17)


['It absorbed over-',
 'whelming light',
 'energy. It fires',
 'lasers that af-',
 'fect every living',
 'being']

In [115]:
lineifyDexEntries(printOneInfoForOneMon('dexTexts', 'UNecrozma', 'Name_PartlyCapital'))

['It absorbed over-',
 'whelming light en-',
 'ergy. It fires',
 'lasers that affect',
 'every living being']

In [8]:
#['SPEAROW', 'RATTATA', 'PIDGEY', 'NIDORAN_M', 'MANKEY', 'EEVEE']

usedAlready = ['PIDGEOTTO', 'NIDOKING', 'CLEFABLE', 'SANDSHREW', 'MANKEY', 'EEVEE'] + ['NIDOKING', 'PIDGEOTTO', 'CLEFABLE', 'SANDSLASH', 'RATICATE', 'EEVEE'] + ['PINSIR', 'PORYGONZ', 'KLEAVOR', 'CLOYSTER', 'NINETALES', 'UMBREON'] + ['ANNIHILAPE', 'ALAKAZAM', 'ELECTIVIRE', 'JYNX', 'MAGMORTAR', 'UMBREON'] + ['JOLTEON', 'VAPOREON', 'FLAREON', 'LEAFEON', 'GLACEON', 'UMBREON']

# ['MPIDGEOT', 'MALAKAZAM', 'MSCIZOR', 'MVENUSAUR', 'MCHARZARDY', 'MBLASTOISE']


In [9]:
tuttiFullyEvolved_unused = []
for here in tuttiFullyEvolved:
    if not here in usedAlready:
        tuttiFullyEvolved_unused.append(here)
print(tuttiFullyEvolved_unused)

['BUTTERFREE', 'BEEDRILL', 'WIGGLYTUFF', 'PARASECT', 'VENOMOTH', 'GOLDUCK', 'FARFETCHD', 'DEWGONG', 'MUK', 'HYPNO', 'KINGLER', 'ELECTRODE', 'MAROWAK', 'WEEZING', 'SEAKING', 'DITTO', 'PIDGEOT', 'FEAROW', 'ARBOK', 'RAICHU', 'CROBAT', 'VILEPLUME', 'BELLOSSOM', 'ARCANINE', 'POLIWRATH', 'POLITOED', 'GOLEM', 'RAPIDASH', 'DODRIO', 'HITMONLEE', 'HITMONCHAN', 'HITMONTOP', 'LICKILICKY', 'KANGASKHAN', 'MR_MIME', 'OMASTAR', 'KABUTOPS', 'VICTREEBEL', 'TENTACRUEL', 'SLOWKING', 'TANGROWTH', 'AERODACTYL', 'VENUSAUR', 'CHARIZARD', 'BLASTOISE', 'NIDOQUEEN', 'DUGTRIO', 'PERSIAN', 'SLOWBRO', 'MAGNEZONE', 'GENGAR', 'STEELIX', 'EXEGGUTOR', 'RHYPERIOR', 'BLISSEY', 'KINGDRA', 'STARMIE', 'SCIZOR', 'TAUROS', 'GYARADOS', 'LAPRAS', 'SNORLAX', 'DRAGONITE', 'ESPEON', 'SYLVEON', 'MACHAMP']


In [10]:
print(len(usedAlready))
print(len(tuttiFullyEvolved))
print(len(tuttiFullyEvolved_unused))

30
85
66


In [ ]:
GENGAR
SNORLAX
LAPRAS
STEELIX
DRAGONITE

GYARADOS
STARMIE
EXEGGUTOR
BLISSEY
RHYPERIOR

TAUROS
KINGDRA
MAGNEZONE
AERODACTYL
TANGROWTH

In [ ]:
SCIZOR
BLASTOISE
CHARIZARD
VENUSAUR

# Random teams for Blue

In [17]:
import random

In [1]:
Deboli = ['BUTTERFREE', 'BEEDRILL', 'CLEFABLE', 'WIGGLYTUFF', 'PARASECT', 'VENOMOTH', 'GOLDUCK', 'FARFETCHD', 'DEWGONG', 'MUK', 'HYPNO', 'KINGLER', 'ELECTRODE', 'MAROWAK', 'WEEZING', 'SEAKING', 'DITTO', ]
Medi = ['PIDGEOT', 'RATICATE', 'FEAROW', 'ARBOK', 'RAICHU', 'SANDSLASH', 'NINETALES', 'CROBAT', 'VILEPLUME', 'BELLOSSOM', 'ARCANINE', 'POLIWRATH', 'POLITOED', 'GOLEM', 'RAPIDASH', 'DODRIO', 'HITMONLEE', 'HITMONCHAN', 'HITMONTOP', 'LICKILICKY', 'KANGASKHAN', 'MR_MIME', 'PINSIR', 'OMASTAR', 'KABUTOPS', 'VICTREEBEL', 'TENTACRUEL', 'SLOWKING', 'CLOYSTER', 'TANGROWTH', 'KLEAVOR', 'ELECTIVIRE', 'MAGMORTAR', 'AERODACTYL', ]
Forti = ['VENUSAUR', 'CHARIZARD', 'BLASTOISE', 'NIDOQUEEN', 'NIDOKING', 'DUGTRIO', 'PERSIAN', 'ANNIHILAPE', 'ALAKAZAM', 'SLOWBRO', 'MAGNEZONE', 'GENGAR', 'STEELIX', 'EXEGGUTOR', 'RHYPERIOR', 'BLISSEY', 'KINGDRA', 'STARMIE', 'SCIZOR', 'JYNX', 'TAUROS', 'GYARADOS', 'LAPRAS', 'SNORLAX', 'DRAGONITE', 'VAPOREON', 'JOLTEON', 'FLAREON', 'ESPEON', 'LEAFEON', 'GLACEON', 'SYLVEON', 'PORYGONZ', 'MACHAMP', ]
Mega = ['MVENUSAUR', 'MCHARZARDX', 'MCHARZARDY', 'MBLASTOISE', 'MBEEDRILL', 'MPIDGEOT', 'MALAKAZAM', 'MSLOWBRO', 'MGENGAR', 'MSTEELIX', 'MKANGASKAN', 'MSCIZOR', 'MPINSIR', 'MGYARADOS', 'MARODACTYL', ]
tuttiFullyEvolved = Deboli + Medi + Forti


In [103]:
def printTeamGivenLists(listOfLists, removeLastComma=False, prefix='80, '):
    team = ''
    for listhere in listOfLists:
        memberHere = random.choice(listhere)
        team += prefix + memberHere + ', '
        listhere.remove(memberHere)
    if removeLastComma:
        team = team[:-2]
    return team

In [104]:
#printTeamGivenLists([Deboli_duplicate, Medi_duplicate, Medi_duplicate, Forti_duplicate, Forti_duplicate])
#print(len(Deboli_duplicate))
#print(len(Medi_duplicate))
#print(len(Forti_duplicate))


In [125]:
Deboli_duplicate = Deboli.copy()
Medi_duplicate = Medi.copy()
Forti_duplicate = Forti.copy()
Mega_duplicate = Mega.copy()

for n in range(21):
    if len(Mega_duplicate)==0:
        Mega_duplicate = Mega.copy()
    if n%5==0:
        listOfLists = [Forti_duplicate, Medi_duplicate, Medi_duplicate, Forti_duplicate, Mega_duplicate]
    if n%5==1 or n%5==2:
        listOfLists = [Forti_duplicate, Medi_duplicate, Deboli_duplicate, Forti_duplicate, Mega_duplicate]
    if n%5==3 or n%5==4:
        listOfLists = [Medi_duplicate, Deboli_duplicate, Medi_duplicate, Forti_duplicate, Mega_duplicate]
    print('db $FF, '+printTeamGivenLists(listOfLists)+'85, UMBREON'+', 0')
print('db $FF, '+printTeamGivenLists([Deboli_duplicate, Deboli.copy(), Medi.copy(), Forti.copy(), Mega_duplicate])+'85, UMBREON'+', 0')

# Extra teams
MegaMewtwos = ['MMEWTWOX', 'MMEWTWOY', ]
for n in range(2):
    teamtemp = ''
    for n in range(4):
        teamtemp += printTeamGivenLists([Mega_duplicate])
    mmewtwotemp = random.choice(MegaMewtwos)
    MegaMewtwos.remove(mmewtwotemp)
    teamtemp += '80, ' + mmewtwotemp + ', '
    teamtemp += '85, UMBREON'
    print('db $FF, '+teamtemp+', 0')
    
teamtemp = ''
leftoversmegas = ['MBEEDRILL', 'MARODACTYL', 'MKANGASKAN', 'MPINSIR', 'MGYARADOS']
for n in range(5):
    teamtemp += printTeamGivenLists([leftoversmegas])
teamtemp += '85, UMBREON'
print('db $FF, '+teamtemp+', 0')
    
#print('db $FF, '+'80, MBEEDRILL, 80, MARODACTYL, 80, MKANGASKAN, 80, MPINSIR, 80, MGYARADOS, 85, UMBREON'+', 0')

db $FF, 80, JYNX, 80, SANDSLASH, 80, FEAROW, 80, CHARIZARD, 80, MPIDGEOT, 85, UMBREON, 0
db $FF, 80, STEELIX, 80, ARCANINE, 80, MUK, 80, RHYPERIOR, 80, MBEEDRILL, 85, UMBREON, 0
db $FF, 80, BLISSEY, 80, HITMONLEE, 80, MAROWAK, 80, SYLVEON, 80, MGYARADOS, 85, UMBREON, 0
db $FF, 80, ELECTIVIRE, 80, SEAKING, 80, SLOWKING, 80, PERSIAN, 80, MALAKAZAM, 85, UMBREON, 0
db $FF, 80, NINETALES, 80, GOLDUCK, 80, KANGASKHAN, 80, JOLTEON, 80, MSTEELIX, 85, UMBREON, 0
db $FF, 80, LAPRAS, 80, CROBAT, 80, AERODACTYL, 80, GENGAR, 80, MPINSIR, 85, UMBREON, 0
db $FF, 80, STARMIE, 80, GOLEM, 80, KINGLER, 80, ANNIHILAPE, 80, MARODACTYL, 85, UMBREON, 0
db $FF, 80, NIDOQUEEN, 80, HITMONCHAN, 80, BEEDRILL, 80, LEAFEON, 80, MCHARZARDX, 85, UMBREON, 0
db $FF, 80, KABUTOPS, 80, DITTO, 80, KLEAVOR, 80, DUGTRIO, 80, MCHARZARDY, 85, UMBREON, 0
db $FF, 80, VILEPLUME, 80, BUTTERFREE, 80, OMASTAR, 80, SCIZOR, 80, MGENGAR, 85, UMBREON, 0
db $FF, 80, MACHAMP, 80, POLIWRATH, 80, MAGMORTAR, 80, GLACEON, 80, MKANGASKAN, 85,

In [96]:
print(len(Deboli_duplicate))
print(len(Medi_duplicate))
print(len(Forti_duplicate))
print(len(Mega_duplicate))

0
0
0
0


In [ ]:
Deboli = ['BUTTERFREE', 'BEEDRILL', 'CLEFABLE', 'WIGGLYTUFF', 'PARASECT', 'VENOMOTH', 'GOLDUCK', 'FARFETCHD', 'DEWGONG', 'MUK', 'HYPNO', 'KINGLER', 'ELECTRODE', 'MAROWAK', 'WEEZING', 'SEAKING', 'DITTO', ]
Medi = ['PIDGEOT', 'RATICATE', 'FEAROW', 'ARBOK', 'RAICHU', 'SANDSLASH', 'NINETALES', 'CROBAT', 'VILEPLUME', 'BELLOSSOM', 'ARCANINE', 'POLIWRATH', 'POLITOED', 'GOLEM', 'RAPIDASH', 'DODRIO', 'HITMONLEE', 'HITMONCHAN', 'HITMONTOP', 'LICKILICKY', 'KANGASKHAN', 'MR_MIME', 'PINSIR', 'OMASTAR', 'KABUTOPS', 'VICTREEBEL', 'TENTACRUEL', 'SLOWKING', 'CLOYSTER', 'TANGROWTH', 'KLEAVOR', 'ELECTIVIRE', 'MAGMORTAR', 'AERODACTYL', ]
Forti = ['VENUSAUR', 'CHARIZARD', 'BLASTOISE', 'NIDOQUEEN', 'NIDOKING', 'DUGTRIO', 'PERSIAN', 'ANNIHILAPE', 'ALAKAZAM', 'SLOWBRO', 'MAGNEZONE', 'GENGAR', 'STEELIX', 'EXEGGUTOR', 'RHYPERIOR', 'BLISSEY', 'KINGDRA', 'STARMIE', 'SCIZOR', 'JYNX', 'TAUROS', 'GYARADOS', 'LAPRAS', 'SNORLAX', 'DRAGONITE', 'VAPOREON', 'JOLTEON', 'FLAREON', 'ESPEON', 'LEAFEON', 'GLACEON', 'SYLVEON', 'PORYGONZ', 'MACHAMP', ]


In [ ]:
'BUTTERFREE', 'BEEDRILL', 'CLEFABLE', 'WIGGLYTUFF', 'PARASECT', 'VENOMOTH', 'GOLDUCK', 'FARFETCHD', 'DEWGONG', 'MUK', 'HYPNO', 'KINGLER', 
'ELECTRODE', 'MAROWAK', 'WEEZING', 'SEAKING', 'DITTO', 'PIDGEOT', 'RATICATE', 'FEAROW', 'ARBOK', 'RAICHU', 'SANDSLASH', 'NINETALES', 'CROBAT', 
'VILEPLUME', 'BELLOSSOM', 'ARCANINE', 'POLIWRATH', 'POLITOED', 'GOLEM', 'RAPIDASH', 'DODRIO', 'HITMONLEE', 'HITMONCHAN', 'HITMONTOP', 'LICKILICKY', 
'KANGASKHAN', 'MR_MIME', 'PINSIR', 'OMASTAR', 'KABUTOPS', 'VICTREEBEL', 'TENTACRUEL', 'SLOWKING', 'CLOYSTER', 'TANGROWTH', 'KLEAVOR', 'ELECTIVIRE', 
'MAGMORTAR', 'AERODACTYL', 'VENUSAUR', 'CHARIZARD', 'BLASTOISE', 'NIDOQUEEN', 'NIDOKING', 'DUGTRIO', 'PERSIAN', 'ANNIHILAPE', 'ALAKAZAM', 'SLOWBRO', 
'MAGNEZONE', 'GENGAR', 'STEELIX', 'EXEGGUTOR', 'RHYPERIOR', 'BLISSEY', 'KINGDRA', 'STARMIE', 'SCIZOR', 'JYNX', 'TAUROS', 'GYARADOS', 'LAPRAS', 
'SNORLAX', 'DRAGONITE', 'VAPOREON', 'JOLTEON', 'FLAREON', 'ESPEON', 'UMBREON', 'LEAFEON', 'GLACEON', 'SYLVEON', 'PORYGONZ', 'MACHAMP', 


In [ ]:
BUTTERFREE
CLEFABLE
HYPNO
   SEAKING
    
---

ARCANINE
TANGROWTH
KLEAVOR
ELECTIVIRE
MAGMORTAR
   POLIWRATH
   POLITOED

---

PERSIAN
ANNIHILAPE
EXEGGUTOR
SCIZOR
   OMASTAR
   KABUTOPS

---

JOLTEON
FLAREON
ESPEON
UMBREON
LEAFEON
GLACEON
SYLVEON
   KINGDRA

In [ ]:
ELECTRODE*
MAROWAK*
SNORLAX*
PORYGONZ

In [33]:
lineifyDialogue("It's the inside of PROF. ELM's LAB!", firstLine=True)
print("")
lineifyDialogue("It looks chaotic, busy, and a bit old-style. In a word, scientific!")
print("")
lineifyDialogue("")
print("")
lineifyDialogue("")
print("")
lineifyDialogue("")
print("")
lineifyDialogue("")
print("")


text "It's the inside of"
line "PROF. ELM's LAB!"

para "It looks chaotic,"
line "busy, and a bit"
cont "old-style. In a"
cont "word, scientific!"







In [ ]:
    db MASTER_BALL
    db ULTRA_BALL
    db GREAT_BALL
    db POKE_BALL
    db MOON_STONE
    db ANTIDOTE
    db BURN_HEAL
    db ICE_HEAL
    db AWAKENING
    db PARLYZ_HEAL
    db FULL_RESTORE
    db MAX_POTION
    db HYPER_POTION
    db SUPER_POTION
    db POTION
    db ESCAPE_ROPE
    db REPEL
    db FIRE_STONE
    db THUNDER_STONE
    db WATER_STONE
    db HP_UP
    db PROTEIN
    db IRON
    db CARBOS
    db CALCIUM
    db RARE_CANDY
    db X_ACCURACY
    db LEAF_STONE
    db NUGGET
    db POKE_DOLL
    db FULL_HEAL
    db REVIVE
    db MAX_REVIVE
    db GUARD_SPEC
    db SUPER_REPEL
    db MAX_REPEL
    db DIRE_HIT
    db FRESH_WATER
    db SODA_POP
    db LEMONADE
    db X_ATTACK
    db X_DEFEND
    db X_SPEED
    db X_SPECIAL
    db PP_UP
    db ETHER
    db MAX_ETHER
    db ELIXER
    db MAX_ELIXER
    db MEGA_STONE_X
    db MEGA_STONE_Y
    db SUN_STONE
    db ICE_STONE
    db KINGS_ROCK
    db METAL_COAT
    db UPGRADE
    db DUBIOUS_DISK
    db FAST_BALL
    db HEAVY_BALL
    db LEGEND_CANDY
    db BIG_NUGGET
    db PERFECTER
    db LIGHT_BALL
    db CHROMOGENE
    db GUTSCHEIN
    db ONIGIRI_BOX
    db OLD_AMBER
    db DOME_FOSSIL
    db HELIX_FOSSIL


In [30]:
numb = 0
for temp in [6,6,5,5,5,
     5,4,4,4,3,
     3,3,3,2,2]:
    numb += temp
    print(hex(numb)[2:].upper())


6
C
11
16
1B
20
24
28
2C
2F
32
35
38
3A
3C


In [26]:
sum([1,2,4,7,10,15,21])


60

In [15]:
lineifyDialogue('10% chance to FLINCH the opponent', lengthMax1=18, lengthMax2=17, firstLine=True,)

text "10% chance to"
line "FLINCH the"
cont "opponent"
